# Searching for real and false stereo

Building on a [previous](https://up42.com/blog/tech/data-ordering101) article, this notebook provides a recipe for searching and ordering sterero data from UP42 using the UP42 [Python SDK](https://sdk.up42.com).

 1. Nutshell definition of what stereo/tristereo is.  
 2. Search for stereo/tristereo.
 3. Order stereo/tristereo.
 4. Get a webhook notification for the order delivery.

**N.B.**: Make sure you are using an SDK version **$\ge$ 0.23.0**. This is the version where support for data products was introduced.

# Installing the UP42 Python SDK

The module is called `up42`. Python **>= 3.8** is **required**.

 1. Create a virtual environment.
```bash
 mkvirtualenv --python=$(which python3) up42-py
```
 2. Activate the environment.
```bash
 workon up42-py
```
 3. Install the module.
 ```bash
 pip install up42-py
 ```
 4. Install Jupyter Lab.
```bash
 pip install jupyterlab
 ```
 5. Done.
 
Now we can just import it.

## Convenience function(s)

Just to make JSON more readable.

In [24]:
from IPython.display import JSON, DisplayObject

def ppjson(json_data: dict, expand:bool=False) -> DisplayObject:
    """Pretty print JSON data."""
    return JSON(json_data, expanded=expand)

In [1]:
import up42

## Authenticate with project ID and project key
The project API key and the project ID are read from a JSON file. This file has the following structure:
```
{
  "project_id": "<your-project-ID-here>",
  "project_api_key": "<your-api-key-here>"
}
```
To find out how to obtain the project API key and project ID the please refer to the [documentation](https://docs.up42.com/processing-platform/projects#developers)

Reading the credentials from the JSON file `~/.up42/proj_default.json`.

In [2]:
from pathlib import Path

project_config_file = Path.home() / ".up42" / "proj_default.json"
up42.authenticate(cfg_file=project_config_file)

2023-06-25 18:07:56,355 - Got credentials from config file.
2023-06-25 18:07:57,140 - Authentication with UP42 successful!


## Stereo and Tristereo in a nutshell

### Stereo basic concept

**Stereo** in the context of satellite imaging refers to the technique of doing two (or three for tristereo) image captures that are shifted by a certain angle among them over a given scene. The value of this shift determines the level of detail that can be reconstructed from combining the two images. Usually these images are used to create, for example, Digital Elevation Models (DEMs), since the two images when properly combined provide _depth_ information, the relief of the scene being imaged can be determined up to a certain degree of certainty.

### Tristereo basic concept

**Tristereo** adds a third image that is captured **midway** between the two extreme positions of a stereo capture. This corresponds to the satellite being vertically aligned with the scene being imaged. The third photo provides additional detail, including making visible relief details that would be hidden when the photo is taken obliquely, which is the case for stereo.

### Satellite orbits and stereo capture

In this notebook, and in fact for most _modern_ earth observation satellites, these two (ot three) image captures are done in a single orbit pass, be it ascending or descending. This is called **along-track** stereo/tristereo.

## Searching for stero/tristero data

### AOI definition

We now look for existing stereo/tristereo data in the region we are interested in. The AOI is defined via a GeoJSON file in the `examples` directory. Let us use [ipyleaflet](https://ipyleaflet.readthedocs.io/) to visualize it.

We are going to look into to two different AOIs:

 1. To look for SPOT 6/7 stereo image sets.
 2. To look for Pléiades 1A/1B tristereo image sets.

Let us start with the stereo image set.

In [3]:
from ipyleaflet import Map, GeoJSON

In [4]:
# Path to the AOI GeoJSON.
path2aoi = "../examples/chuquicamata.geojson"

In [5]:
import json

with open(path2aoi, "r") as f:
    aoi_map = json.load(f)

In [7]:
%pip install toolz

  Using cached toolz-0.12.0-py3-none-any.whl (55 kB)

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from toolz.dicttoolz import get_in

In [10]:
from shapely.geometry import shape

In [13]:
aoi_map_shape = shape(get_in(["features", 0, "geometry"], aoi_map))

In [20]:
map_center = aoi_map_shape.centroid.coords[0][::-1]

In [21]:
mymap = Map(center=map_center, zoom=14)

# Add the AOI to the map. First style it and then add it.
aoi_layer = GeoJSON(
    data=aoi_map,
    style={"opacity": 1, "dashArray": "9", "fillOpacity": 0.5, "weight": 1},
    hover_style={"color": "yellow", "dashArray": "0", "fillOpacity": 0.5},
)
mymap.add_layer(aoi_layer)
mymap

Map(center=[-22.33093995821105, -68.90701304941625], controls=(ZoomControl(options=['position', 'zoom_in_text'…

### Searching for data in the catalog
We need to instantiate a catalog object and perform a search.

In [22]:
catalog = up42.initialize_catalog()

#### Getting the display product ID

Before performing the search we need to select the data product we are interested in. In this case we are interested in a display product of a Pléiades image. The data product we want is not necessary for the search itself, but we do it now so that the ordering flow stays simple. Since we do the data product selection at the outset and not after performing the search. This reproduces the usual actions of having the product specified before hand and then performing the search.

In [25]:
products = catalog.get_data_products(basic=True)
ppjson(products)

<IPython.core.display.JSON object>

We have the list of all **available** collections and the corresponding products. In our case we want SPOT 6/7.

Specifically we want a display product. We need to get the corresponding product ID.

In [26]:
pleiades_product = next(
    filter(lambda e: e[0].lower().endswith("ades"), products.items())
)
pleiades_product

('Pléiades',
 {'collection': 'phr',
  'host': 'oneatlas',
  'data_products': {'Analytic': '4f1b2f62-98df-4c74-81f4-5dce45deee99',
   'Display': '647780db-5a06-4b61-b525-577a8b68bb54'}})

In [43]:
pleiades_product_id = get_in([1, "data_products", "Display"], pleiades_product)
pleiades_product_id

'647780db-5a06-4b61-b525-577a8b68bb54'

And the collection name.

In [42]:
pleiades_coll_name = get_in([1, "collection"], pleiades_product)
pleiades_coll_name

'phr'

Let us now perform the search.

We are interested in imagery between April 1st 2021 and June 26th, 2023.

In [31]:
my_start_date = "2021-04-01"
my_end_date = "2023-06-26"

In [33]:
pleiades_search_params = catalog.construct_search_parameters(
    geometry=aoi_map,
    start_date=my_start_date,
    end_date=my_end_date,
    collections=[pleiades_coll_name],
    max_cloudcover=10,
    sortby="acquisitionDate",
    ascending=False,
    limit=50,
)

In [34]:
pleiades_search_results = catalog.search(pleiades_search_params, as_dataframe=False)

2023-06-25 18:29:01,629 - Searching catalog with search_parameters: {'datetime': '2021-04-01T00:00:00Z/2023-06-26T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((-68.92192558463671, -22.32018472450315), (-68.9242442302506, -22.341155732344163), (-68.8877470307674, -22.341155732344163), (-68.89496059490037, -22.31915197980001), (-68.92192558463671, -22.32018472450315)),)}, 'limit': 50, 'collections': ['phr'], 'query': {'cloudCoverage': {'lte': 10}}, 'sortby': [{'field': 'properties.acquisitionDate', 'direction': 'desc'}]}
2023-06-25 18:29:04,109 - 14 results returned.


Let us look for the stereo pairs in the above search results.

In [37]:
# N.B: This is an ugly hack that needs to be replaced by creating a proper local Python package for the Notebook helpers.
import sys
sys.path.append(str((Path.cwd() / ".." / "..").resolve()))

In [38]:
from pynb_helpers import stereo

In [55]:
pleiades_stereo_results = stereo.select_stereo(
    pleiades_search_results["features"]
)
ppjson(pleiades_stereo_results)

<IPython.core.display.JSON object>

We need to get the image IDs for placing the order. We use the *convenience* function `get_tristereo_image_ids`.

We can even look for tristereo.

In [54]:
pleiades_tristereo_results = stereo.select_tristereo(pleiades_search_results["features"])
ppjson(pleiades_tristereo_results)

<IPython.core.display.JSON object>

## Looking for false stereo

We already know from above that there are no stereo captures between April 1st, 2021 and Feb 20th 2022. So let us try our luck regarding false stereo for this time range.

In [97]:
my_false_stereo_end_date="2020-01-01"
my_false_stereo_start_date="2019-10-01"

Doing a search just for this date range.

In [98]:
false_stereo_search_params = catalog.construct_search_parameters(
    geometry=aoi_map,
    start_date=my_false_stereo_start_date,
    end_date=my_false_stereo_end_date,
    collections=[pleiades_coll_name],
    max_cloudcover=10,
    sortby="acquisitionDate",
    ascending=False,
    limit=50,
)

In [99]:
false_stereo_search_results = catalog.search(false_stereo_search_params, as_dataframe=False)

2023-06-26 12:17:32,182 - Searching catalog with search_parameters: {'datetime': '2019-10-01T00:00:00Z/2020-01-01T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((-68.92192558463671, -22.32018472450315), (-68.9242442302506, -22.341155732344163), (-68.8877470307674, -22.341155732344163), (-68.89496059490037, -22.31915197980001), (-68.92192558463671, -22.32018472450315)),)}, 'limit': 50, 'collections': ['phr'], 'query': {'cloudCoverage': {'lte': 10}}, 'sortby': [{'field': 'properties.acquisitionDate', 'direction': 'desc'}]}
2023-06-26 12:17:34,796 - 15 results returned.


We are interested only in images were the incidence angle is **below 30 &deg;**.

In [120]:
filtered_false_stereo_search_results =list(
    filter(lambda f: get_in(["properties", "providerProperties", "incidenceAngle"], f) < 30, 
           false_stereo_search_results["features"]))

Now we need to define the time difference in seconds so that we can look for the false stereo.

In [74]:
false_stereo_delay = stereo.compute_delay(my_false_stereo_start_date,  my_false_stereo_end_date)
false_stereo_delay

31536000

In [116]:
false_stereo_candidates = stereo.select_stereo(filtered_false_stereo_search_results, false_stereo_delay)
ppjson(false_stereo_candidates)

<IPython.core.display.JSON object>

## Obtaining primary products

Currently it is not possible to obtain primary products via API from Airbus. If you want to create a DEM then we can obtain these products for you. 

 1. From the process described in the notebook you can get the information you need to place the order.
 2. Send your request via email to [ordering@up42.com](mailto:ordering@up42.com).

In [121]:
pleiades_search_params

{'datetime': '2021-04-01T00:00:00Z/2023-06-26T23:59:59Z',
 'intersects': {'type': 'Polygon',
  'coordinates': (((-68.92192558463671, -22.32018472450315),
    (-68.9242442302506, -22.341155732344163),
    (-68.8877470307674, -22.341155732344163),
    (-68.89496059490037, -22.31915197980001),
    (-68.92192558463671, -22.32018472450315)),)},
 'limit': 50,
 'collections': ['phr'],
 'query': {'cloudCoverage': {'lte': 10}},
 'sortby': [{'field': 'properties.acquisitionDate', 'direction': 'desc'}]}